In [1]:
import numpy as np
import xarray as xr
import pandas as pd
import numcodecs
from dask.distributed import Client, progress, LocalCluster
import zarr
import glob
from datetime import datetime
import time
import s3fs

In [2]:
cluster = LocalCluster(n_workers=4)
client = Client(cluster)
print(f"Dask client {client}")

Dask client <Client: 'tcp://127.0.0.1:33511' processes=4 threads=64, memory=535.16 GB>


In [3]:
def generate_file_list(start_doy, end_doy, year):   
    """
    Given a start day and end end day, generate a list of file locations.
    Assumes a 'prefix' and 'year' variables have already been defined.
    'Prefix' should be a local directory or http url and path.
    'Year' should be a 4 digit year.
    """
    days_of_year = list(range(start_doy, end_doy))
    fileObjs = []
    for doy in days_of_year:
        if doy < 10:
            doy = f"00{doy}"
        elif doy >= 10 and doy < 100:
            doy = f"0{doy}"            
        file = glob.glob(f"{netcdf_prefix}/{year}/{doy}/*.nc")[0]
        fileObjs.append(file)
    return fileObjs

In [4]:
# Invariants - but should be made configurable
netcdf_prefix = f"/s3fsx/eodc/mursst_netcdf"
chunks = {'time': 5, 'lat': 1799, 'lon': 3600}
path = 'x'.join(map(str, chunks.values()))
# CLI Argument - zarr directory
store_dir = f"/s3fsx/eodc/mursst_zarr/5x1799x3600"
numcodecs.blosc.use_threads = False
print(f"zarr store directory: {store_dir}")

zarr store directory: /s3fsx/eodc/mursst_zarr/5x1799x3600


In [9]:
%%time
ds_zarr = xr.open_zarr(store_dir, consolidated=True, mask_and_scale=False)
ds_zarr

distributed.utils_perf - WARNING - full garbage collections took 33% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 33% CPU time recently (threshold: 10%)


CPU times: user 3.47 s, sys: 140 ms, total: 3.61 s
Wall time: 6.25 s


<xarray.Dataset>
Dimensions:           (lat: 17999, lon: 36000, time: 5328)
Coordinates:
  * lat               (lat) float32 -89.99 -89.98 -89.97 ... 89.97 89.98 89.99
  * lon               (lon) float32 -179.99 -179.98 -179.97 ... 179.99 180.0
  * time              (time) datetime64[ns] 2002-06-01T09:00:00 ... 2016-12-31T09:00:00
Data variables:
    analysed_sst      (time, lat, lon) int16 dask.array<chunksize=(5, 1799, 3600), meta=np.ndarray>
    analysis_error    (time, lat, lon) int16 dask.array<chunksize=(5, 1799, 3600), meta=np.ndarray>
    mask              (time, lat, lon) int8 dask.array<chunksize=(5, 1799, 3600), meta=np.ndarray>
    sea_ice_fraction  (time, lat, lon) int8 dask.array<chunksize=(5, 1799, 3600), meta=np.ndarray>
Attributes:
    Conventions:                CF-1.5
    Metadata_Conventions:       Unidata Observation Dataset v1.0
    acknowledgment:             Please acknowledge the use of these data with...
    cdm_data_type:              grid
    comment:                    MUR = "Multi-scale Ultra-high Reolution"
    creator_email:              ghrsst@podaac.jpl.nasa.gov
    creator_name:               JPL MUR SST project
    creator_url:                http://mur.jpl.nasa.gov
    date_created:               20170104T014705Z
    easternmost_longitude:      180.0
    file_quality_level:         1
    gds_version_id:             2.0
    geospatial_lat_resolution:  0.01 degrees
    geospatial_lat_units:       degrees north
    geospatial_lon_resolution:  0.01 degrees
    geospatial_lon_units:       degrees east
    history:                    created at nominal 4-day latency; replaced nr...
    id:                         MUR-JPL-L4-GLOB-v04.1
    institution:                Jet Propulsion Laboratory
    keywords:                   Oceans > Ocean Temperature > Sea Surface Temp...
    keywords_vocabulary:        NASA Global Change Master Directory (GCMD) Sc...
    license:                    These data are available free of charge under...
    metadata_link:              http://podaac.jpl.nasa.gov/ws/metadata/datase...
    naming_authority:           org.ghrsst
    netcdf_version_id:          4.1
    northernmost_latitude:      90.0
    platform:                   Aqua, DMSP, NOAA-POES, Suomi-NPP, Terra
    processing_level:           L4
    product_version:            04.1
    project:                    NASA Making Earth Science Data Records for Us...
    publisher_email:            ghrsst-po@nceo.ac.uk
    publisher_name:             GHRSST Project Office
    publisher_url:              http://www.ghrsst.org
    references:                 http://podaac.jpl.nasa.gov/Multi-scale_Ultra-...
    sensor:                     AMSR-E, AVHRR, MODIS, SSM/I, VIIRS, in-situ
    source:                     AVHRR19_G-NAVO, AVHRR_METOP_A-EUMETSAT, MODIS...
    southernmost_latitude:      -90.0
    spatial_resolution:         0.01 degrees
    standard_name_vocabulary:   NetCDF Climate and Forecast (CF) Metadata Con...
    start_time:                 20161231T090000Z
    stop_time:                  20161231T090000Z
    summary:                    A merged, multi-sensor L4 Foundation SST anal...
    time_coverage_end:          20161231T210000Z
    time_coverage_start:        20161230T210000Z
    title:                      Daily MUR SST, Final product
    uuid:                       27665bc0-d5fc-11e1-9b23-0800200c9a66
    westernmost_longitude:      -180.0

In [8]:
%%time
# Loop and append
# Command Line Argument - year, start_day, number_batches_to_append, batch_size
year = 2016
start_doy = 366
end_doy = start_doy
number_batches_to_append = 1
batch_size = 5
final_end_doy = start_doy + (number_batches_to_append * batch_size)

while start_doy < final_end_doy:
    end_doy = start_doy + batch_size
    end_doy = min(367, end_doy)
    fileObjs = generate_file_list(start_doy, end_doy, year)
    first_file = fileObjs[0].split('/')[-1]
    last_file = fileObjs[-1].split('/')[-1]
    print(f"start doy: {start_doy}, starting file: {first_file}")
    print(f"end doy: {end_doy}, ending file: {last_file}")
    args = {'consolidated': True}
    # Either append or initiate store
    if start_doy == 152 and year == 2002:
        ds = xr.open_mfdataset(fileObjs, parallel=True, combine='by_coords', mask_and_scale=False)
        ds = ds.chunk(chunks)       
        args['mode'] = 'w'
    else:
        # Check here that the next day we will append is the next day in the year
        current_ds = xr.open_zarr(store_dir, consolidated=True)
        next_day = current_ds.time[-1].values + np.timedelta64(1, 'D')
        next_day_str = str(next_day)[0:10].replace('-', '') 
        if not (first_file[0:8] == next_day_str):
            raise Exception("starting file is not the next day of the year")
            break
        drop_vars = ['dt_1km_data', 'sst_anomaly']
        ds = xr.open_mfdataset(fileObjs, parallel=True, combine='by_coords', drop_variables=drop_vars)
        ds = ds.chunk(chunks)        
        args['mode'] = 'a'
        args['append_dim'] = 'time'
    ds.to_zarr(store_dir, **args)
    start_doy = end_doy
    print(f"Done with this batch")
    print()

start doy: 366, starting file: 20161231090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
end doy: 367, ending file: 20161231090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc


distributed.utils_perf - WARNING - full garbage collections took 33% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 33% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 35% CPU time recently (threshold: 10%)


Done with this batch



IndexError: list index out of range

In [10]:
time_slice = slice(datetime.strptime(f"2016-12-21", '%Y-%m-%d'), datetime.strptime(f"2016-12-31", '%Y-%m-%d'))

In [11]:
%%time
fileObjs = generate_file_list(1, 367, 2016)
print(f"starting file: {fileObjs[0].split('/')[-1]}")
print(f"ending file: {fileObjs[-1].split('/')[-1]}")
drop_vars = ['dt_1km_data', 'sst_anomaly']
ds_netcdf = xr.open_mfdataset(fileObjs, parallel=True, combine='by_coords', mask_and_scale=False, drop_variables=drop_vars)
ds_netcdf

starting file: 20160101090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
ending file: 20161231090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
CPU times: user 1.89 s, sys: 760 ms, total: 2.65 s
Wall time: 8.7 s


<xarray.Dataset>
Dimensions:           (lat: 17999, lon: 36000, time: 366)
Coordinates:
  * lat               (lat) float32 -89.99 -89.98 -89.97 ... 89.97 89.98 89.99
  * lon               (lon) float32 -179.99 -179.98 -179.97 ... 179.99 180.0
  * time              (time) datetime64[ns] 2016-01-01T09:00:00 ... 2016-12-31T09:00:00
Data variables:
    analysed_sst      (time, lat, lon) int16 dask.array<chunksize=(1, 17999, 36000), meta=np.ndarray>
    analysis_error    (time, lat, lon) int16 dask.array<chunksize=(1, 17999, 36000), meta=np.ndarray>
    mask              (time, lat, lon) int8 dask.array<chunksize=(1, 17999, 36000), meta=np.ndarray>
    sea_ice_fraction  (time, lat, lon) int8 dask.array<chunksize=(1, 17999, 36000), meta=np.ndarray>
Attributes:
    Conventions:                CF-1.5
    title:                      Daily MUR SST, Final product
    summary:                    A merged, multi-sensor L4 Foundation SST anal...
    references:                 http://podaac.jpl.nasa.gov/Multi-scale_Ultra-...
    institution:                Jet Propulsion Laboratory
    history:                    created at nominal 4-day latency; replaced nr...
    comment:                    MUR = "Multi-scale Ultra-high Reolution"
    license:                    These data are available free of charge under...
    id:                         MUR-JPL-L4-GLOB-v04.1
    naming_authority:           org.ghrsst
    product_version:            04.1
    uuid:                       27665bc0-d5fc-11e1-9b23-0800200c9a66
    gds_version_id:             2.0
    netcdf_version_id:          4.1
    date_created:               20180727T190247Z
    start_time:                 20160101T090000Z
    stop_time:                  20160101T090000Z
    time_coverage_start:        20151231T210000Z
    time_coverage_end:          20160101T210000Z
    file_quality_level:         1
    source:                     MODIS_T-JPL, MODIS_A-JPL, AMSR2-REMSS, AVHRR1...
    platform:                   Terra, Aqua, GCOM-W, NOAA-19, MetOp-A, Buoys/...
    sensor:                     MODIS, AMSR2, AVHRR, in-situ
    Metadata_Conventions:       Unidata Observation Dataset v1.0
    metadata_link:              http://podaac.jpl.nasa.gov/ws/metadata/datase...
    keywords:                   Oceans > Ocean Temperature > Sea Surface Temp...
    keywords_vocabulary:        NASA Global Change Master Directory (GCMD) Sc...
    standard_name_vocabulary:   NetCDF Climate and Forecast (CF) Metadata Con...
    southernmost_latitude:      -90.0
    northernmost_latitude:      90.0
    westernmost_longitude:      -180.0
    easternmost_longitude:      180.0
    spatial_resolution:         0.01 degrees
    geospatial_lat_units:       degrees north
    geospatial_lat_resolution:  0.01 degrees
    geospatial_lon_units:       degrees east
    geospatial_lon_resolution:  0.01 degrees
    acknowledgment:             Please acknowledge the use of these data with...
    creator_name:               JPL MUR SST project
    creator_email:              ghrsst@podaac.jpl.nasa.gov
    creator_url:                http://mur.jpl.nasa.gov
    project:                    NASA Making Earth Science Data Records for Us...
    publisher_name:             GHRSST Project Office
    publisher_url:              http://www.ghrsst.org
    publisher_email:            ghrsst-po@nceo.ac.uk
    processing_level:           L4
    cdm_data_type:              grid

In [12]:
%%time
print(ds_netcdf.where(ds_netcdf.analysed_sst != -32768).analysed_sst[356:366,:,:].sel(lat=slice(40,50),lon=slice(-170,-160)).mean().values)
ds_netcdf.where(ds_netcdf.analysed_sst != -32768).analysed_sst.sel(time=time_slice).mean().values

-17803.61192264279
CPU times: user 973 ms, sys: 155 ms, total: 1.13 s
Wall time: 38.9 s


array(-11290.59658304)

In [13]:
%%time
print(ds_zarr.where(ds_zarr.analysed_sst != -32768).analysed_sst[5318:5328,:,:].sel(lat=slice(40,50),lon=slice(-170,-160)).mean().values)
ds_zarr.where(ds_zarr.analysed_sst != -32768).analysed_sst.sel(time=time_slice).mean().values

-17803.61192264279


distributed.utils_perf - WARNING - full garbage collections took 32% CPU time recently (threshold: 10%)


CPU times: user 10.8 s, sys: 481 ms, total: 11.3 s
Wall time: 28.1 s


array(-11290.59658304)

In [ ]:
import s3fs

s3 = s3fs.S3FileSystem(anon=True, client_kwargs=dict(region_name='us-east-1'))
s3_store = s3fs.S3Map(root='nasa-eodc/eodc/mursst_zarr/5x1799x3600', s3=s3, check=False)
ds_zarr = xr.open_zarr(s3_store, consolidated=True, mask_and_scale=False)

In [ ]:
%%time
print(ds_zarr.where(ds_zarr.analysed_sst != -32768).analysed_sst[4577:4597,:,:].sel(lat=slice(40,50),lon=slice(-170,-160)).mean().values)
ds_zarr.where(ds_zarr.analysed_sst != -32768).analysed_sst.sel(time=time_slice).mean().values

In [14]:
!ls /s3fsx/eodc/mursst_netcdf/2017/**/*.nc | wc -l

11
